"""
This file extracts and handles DOL data
"""

In [120]:
import os, sys, time
working_dir = 'H:/UGA MASTERS/SDG/DOL'
os.chdir(working_dir)

import pandas as pd
import sys

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [152]:
"""
convert money to float
"""
from re import sub
def money2float(x):
    return float(sub(r'[^\d.]', '', x))

"""
group set of joe titles
"""
def get_subset(df, names):
    # yearly salarys only
    df = df[df['PW_UNIT_OF_PAY']=='Year']
    return df[df['JOB_TITLE_CLEAN'].isin(names)]

"""
plot salary
"""
import plotly.express as px
def plot_salary(fig, df, job_name, trace_name):
    fig.add_trace(go.Histogram(x=df['PAY'], name=trace_name))
    fig.update_layout(title_text=f'Salary for {job_name}',)
    fig.show()
    
def getMedianByJob(all_df, names, state=None, city=None):
    sub_df = get_subset(all_df, names)
    
    if state != None:
        sub_df = sub_df[sub_df['EMPLOYER_STATE']==state]
    if city != None:
        sub_df = sub_df[sub_df['EMPLOYER_CITY']==city]
        
    print(sub_df['PAY'].median())
    return sub_df['PAY'].median()

def cleanJobs(x):
    x = str(x)
    x = x.lower()
    return x

def getAllNameVariants(all_df, job_name):
    names = []
    for name in all_df['JOB_TITLE_CLEAN'].value_counts().index:
        if job_name in name:
            names.append(name)
    return names

In [136]:
"""
load all quarters
"""
df1 = pd.read_csv('data/LCA_Disclosure_Data_FY2020_Q1.csv', low_memory=False)
df2 = pd.read_csv('data/LCA_Disclosure_Data_FY2020_Q2.csv', low_memory=False)
df3 = pd.read_csv('data/LCA_Disclosure_Data_FY2020_Q3.csv', low_memory=False)
df4 = pd.read_csv('data/LCA_Disclosure_Data_FY2020_Q4.csv', low_memory=False)

list_of_dataframe = [df1, df2, df3, df4] #concat quarters 
all_df = pd.concat(list_of_dataframe)
all_df['PAY'] = all_df['WAGE_RATE_OF_PAY_FROM'].map(money2float) # convert pay
all_df['JOB_TITLE_CLEAN'] = all_df['JOB_TITLE'].map(cleanJobs)

In [161]:
ds_names = getAllNameVariants(all_df, 'data scientist')
de_names = getAllNameVariants(all_df, 'data engineer')

print('Data Engineer Queries')
val = getMedianByJob(all_df, de_names)
val = getMedianByJob(all_df, de_names, state='CA')
val = getMedianByJob(all_df, de_names, state='CA', city='IRVINE')
val = getMedianByJob(all_df, de_names, state='CA', city='PALO ALTO')

print('Data Scientist Queries')
val = getMedianByJob(all_df, ds_names)
val = getMedianByJob(all_df, ds_names, state='CA')
val = getMedianByJob(all_df, ds_names, state='CA', city='IRVINE')
val = getMedianByJob(all_df, ds_names, state='CA', city='PALO ALTO')

Data Engineer Queries
110469.0
140662.0
117686.4
145000.0
Data Scientist Queries
113300.0
137000.0
105960.0
104000.0


In [155]:
fig = go.Figure()
plot_salary(fig, de_df, 'Data Engineer', 'Data Engineer')
plot_salary(fig, ds_df, 'Data Scientist', 'Data Scientist')

In [ ]:
"""
Explore all job titles:

load all job title names by order of popularity
"""
for name in all_df['JOB_TITLE_CLEAN'].value_counts().index:
    name_low = name.lower()
    if 'data' in name_low:
        print(name)

de_names = []
for name in all_df['JOB_TITLE_CLEAN'].value_counts().index:
    if 'data engineer' in name:
        print(name)
        de_names.append(name)
        
ds_names = []
for name in all_df['JOB_TITLE_CLEAN'].value_counts().index:
    if 'data scientist' in name:
        print(name)
        ds_names.append(name)

In [156]:
# fig = go.Figure()
# plot_salary(fig, de_df, 'Data Engineer', 'Total')

# de_df_ca = de_df[de_df['EMPLOYER_STATE']=='CA']
# plot_salary(fig, de_df_ca, 'Data Engineer','CA')

# # de_df_ca = de_df[de_df['EMPLOYER_STATE']=='GA']
# # plot_salary(fig, de_df_ca, 'Data Engineer','GA')


# fig = go.Figure()
# de_df_ca = de_df[de_df['EMPLOYER_STATE']=='CA']
# plot_salary(fig, de_df_ca, 'Data Engineer','CA')
# de_df_ca = de_df[de_df['EMPLOYER_STATE']=='GA']
# plot_salary(fig, de_df_ca, 'Data Engineer','GA')
# de_df_ca = de_df[de_df['EMPLOYER_STATE']=='WA']
# plot_salary(fig, de_df_ca, 'Data Engineer','WA')